# Drug Name Entity Classifier
## AHLT - MIRI 2018



## Initialization

Load needed modules and specify the working directory

In [1]:
# Load needed packages
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV,KFold # Parameter selection
import time # Execution time of some blocks
from nltk.tag import StanfordPOSTagger
import statistics
import scipy.stats # for RandomizedSearchCV


# Import our defined functions
from drug_functions import *
from makingPredictions import *
from datasetBuilder import *

In [2]:
# init time
first_init = time.time()

In [3]:
# Set the data directories
# train_dirs_whereto_parse = ['data/medium_train_DrugBank']
# test_dirs_whereto_parse = ['data/medium_test_DrugBank']
train_dirs_whereto_parse = ['data/Train/DrugBank','data/Train/MedLine']
test_dirs_whereto_parse = ['data/Test/DrugBankOutput','data/Test/MedlineOutput']

## Reading the train and test data from the XML files
Accessing to all the files of the directory and storing id's and text's in two arrays.
We have also added the token 'STOP' at the end of each sentence.

In [4]:
train_texts_entities = createTrainSet(train_dirs_whereto_parse)
test_texts_entities = createTestSet(test_dirs_whereto_parse)

Before computing features, I want the input data to have a special format:

In [5]:
# we want each dataset with the following format: 
# for ['have','Ibuprofeno'] ---- [('hola','V','O'),('Ibuprofeno','NN','B')]

def buildSet(text_entities):
    dataset = []
    for text,drugs in text_entities:
        # tokenizing
        tokenized_sentence = nltk.word_tokenize(text)
        # BIO tagging
        tokens_tags = BIOTagger(text, drugs)
        # POS tagging
        tokens_pos = nltk.pos_tag(tokenized_sentence)

        text_triples = []
        for idx,token in enumerate(tokenized_sentence):
            text_triples.append((token,tokens_pos[idx][1],tokens_tags[idx][1]))
        dataset.append(text_triples)
    
    return dataset

# train_set and test_set are list of list of triples; each list of triples refers to a different sentence
train_set = buildSet(train_texts_entities) 
test_set = buildSet(test_texts_entities)

print('Number of training sentences: ',len(train_set))
print('Num of test sentences: ',len(test_set))

Number of training sentences:  5560
Num of test sentences:  665


## FEATURES 

In [6]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [7]:
 # remember s is a triple defined as follows: (token, POS tag, BIO tag)
X_train = [sent2features(s) for s in train_set]
y_train = [sent2labels(s) for s in train_set]

X_test = [sent2features(s) for s in test_set]
y_test = [sent2labels(s) for s in test_set]

In [8]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

####  Predictions

In [9]:
pred_tags = crf.predict(X_test)


####  Evaluation

In [10]:
precision = []
recall = []
for idx, text_drugs in enumerate(test_texts_entities): 
    # so text_drugs will be a tuple of (sentence, list of true entities in that sentence)
    tokens = nltk.word_tokenize(text_drugs[0])
    predicted_entities = BIOTagsToEntities(tokens,pred_tags[idx])
    precision = precision + [compute_precision(predicted_entities,text_drugs[1])]
    recall = recall + [compute_recall(predicted_entities,text_drugs[1])]

avg_precision = statistics.mean(precision)
avg_recall = statistics.mean(recall)

# F1 metric
F1 = round((2*avg_precision*avg_recall) / (avg_precision + avg_recall),2)
print(F1)

37.07


In [11]:
print('Total execution time: ',(time.time() - first_init), ' seconds')

Total execution time:  24.08883309364319  seconds


In [12]:

'''
## Log of results
date, precision, recall, F1, features, test
14-May, 46.2, 52.1, 48.99, Token length; Prefixes/Suffixes; POS tag; Binary features (+-2); Token position; DrugBank DB; Shape, yes
'''

'\n## Log of results\ndate, precision, recall, F1, features, test\n14-May, 46.2, 52.1, 48.99, Token length; Prefixes/Suffixes; POS tag; Binary features (+-2); Token position; DrugBank DB; Shape, yes\n'